# Time benchmark KDE

In [1]:
from abc import ABC, abstractmethod
from dataclasses import dataclass

import numpy as np
import pandas as pd

import time

from scipy.stats import norm
from scipy.stats import multivariate_normal

from scipy.stats import gaussian_kde
from sklearn.neighbors import KernelDensity
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.nonparametric.kernel_density import KDEMultivariate
from KDEpy.FFTKDE import FFTKDE

import parallelkdepy as pkde

from tqdm.notebook import trange, tqdm

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [2]:
def get_distro(n_dims):
    if n_dims == 1:
        return norm(loc=0.0, scale=1.0)
    elif n_dims == 2:
        return multivariate_normal(mean=np.full(2, 0.0), cov=np.eye(2))
    else:
        raise ValueError("Only 1D and 2D distro allowed")

In [3]:
def sample_distro(distro, n_samples):
    return distro.rvs(size=n_samples)

In [4]:
def create_grid(n_points, dim, lb=-8, hb=8, device="cpu"):
    grid = pkde.Grid(dim*[(lb, hb, n_points)], device=device)

    return grid

In [5]:
@dataclass
class EstimatorInfo:
    package: str
    method: str
    dim: int
    device: str = "cpu"

In [6]:
class DensityEstimator(ABC):
    def __init__(self, info: EstimatorInfo):
        self._info = info
        self._is_fit = False

    @property
    def info(self):
        return self._info

    @abstractmethod
    def fit(self, x: np.ndarray):
        ...

    @abstractmethod
    def evaluate(self, grid) -> np.ndarray:
        ...

    def name(self) -> str:
        return f"{self._info.package}:{self._info.method}:{self._info.device}:{self._info.dim}D"

In [7]:
class ScipyKDE(DensityEstimator):
    def __init__(self, dim: int, bw_method: str):
        super().__init__(EstimatorInfo("scipy", bw_method, dim))

    def fit(self, data: np.ndarray, **kwargs):
        if data.ndim > 1:
            assert data.shape[1] == self.info.dim
        else:
            assert self.info.dim == 1
        self._data = data
        self._kde = gaussian_kde(data.T, bw_method=self.info.method)
        self._is_fit = True
        
        return self

    def evaluate(self, grid: pkde.Grid) -> np.ndarray:
        grid_mesh = grid.to_meshgrid()
        if len(grid_mesh) > 1:
            grid_points = np.vstack([x.ravel() for x in grid_mesh])
        else:
            grid_points = grid_mesh[0]
            
        density_estimated = self._kde(grid_points).reshape(grid.shape)

        return density_estimated

In [8]:
class SklearnKDE(DensityEstimator):
    def __init__(self, dim: int, bw_method: str):
        super().__init__(EstimatorInfo("sklearn", bw_method, dim))

    def fit(self, data: np.ndarray):
        if data.ndim > 1:
            assert data.shape[1] == self.info.dim
            self._data = data
        else:
            assert self.info.dim == 1
            self._data = data[:, np.newaxis]
        self._kde = KernelDensity(bandwidth=self.info.method).fit(self._data)
        self._is_fit = True

        return self

    def evaluate(self, grid: pkde.Grid) -> np.ndarray:
        grid_mesh = grid.to_meshgrid()
        if len(grid_mesh) > 1:
            grid_points = np.vstack([x.ravel() for x in grid_mesh])
            grid_points = grid_points.T
        else:
            grid_points = grid_mesh[0]
            grid_points = grid_points[:, np.newaxis]

        density_estiated = np.exp(self._kde.score_samples(grid_points)).reshape(grid.shape)

        return density_estiated

In [9]:
class StatsmodelsKDE1D(DensityEstimator):
    def __init__(self, bw_method: str):
        super().__init__(EstimatorInfo("statsmodels", bw_method, 1))

    def fit(self, data: np.ndarray):
        if data.ndim > 1:
            assert data.shape[1] == self.info.dim
        else:
            assert self.info.dim == 1
        self._data = data
        self._kde = KDEUnivariate(data)
        self._is_fit = True

        return self

    def evaluate(self, grid: pkde.Grid) -> np.ndarray:
        grid_size = grid.shape[0]
        self._kde.fit(bw=self.info.method, fft=True, gridsize=grid_size)

        grid_points = grid.to_meshgrid()[0]
        density_estimated = self._kde.evaluate(grid_points)

        return density_estimated

In [10]:
class StatsmodelsKDE2D(DensityEstimator):
    def __init__(self, bw_method: str):
        super().__init__(EstimatorInfo("statsmodels", bw_method, 2))

    def fit(self, data: np.ndarray):
        if data.ndim > 1:
            assert data.shape[1] == self.info.dim
        else:
            assert self.info.dim == 1
        self._data = data
        self._kde = KDEMultivariate(data=data, var_type="cc", bw=self.info.method)
        self._is_fit = True

    def evaluate(self, grid: pkde.Grid) -> np.ndarray:
        grid_mesh = grid.to_meshgrid()
        grid_points = np.vstack([x.ravel() for x in grid_mesh])
        
        density_estimated = self._kde.pdf(grid_points).reshape(grid.shape)

        return density_estimated

In [11]:
class KDEpyKDE(DensityEstimator):
    def __init__(self, dim: int, bw_method: str):
        super().__init__(EstimatorInfo("kdepy", bw_method, dim))

    def fit(self, data: np.ndarray):
        if data.ndim > 1:
            assert data.shape[1] == self.info.dim
        else:
            assert self.info.dim == 1
        self._data = data
        self._kde = FFTKDE(bw=self.info.method).fit(data)
        self._is_fit = True

        return self

    def evaluate(self, grid: pkde.Grid) -> np.ndarray:
        grid_mesh = grid.to_meshgrid()
        if len(grid_mesh) > 1:
            grid_points = np.vstack([x.ravel() for x in grid_mesh])
        else:
            grid_points = grid_mesh[0]

        density_estimated = self._kde.evaluate(grid_points).reshape(grid.shape)

        return density_estimated

In [12]:
class ParallelKDE(DensityEstimator):
    def __init__(self, dim:int, bw_method: str, device: str):
        super().__init__(EstimatorInfo("parallelkdepy", bw_method, dim, device))

    def fit(self, data: np.ndarray):
        if data.ndim > 1:
            assert data.shape[1] == self.info.dim
            self._data = data
        else:
            assert self.info.dim == 1
            self._data = data[:, np.newaxis]
        self._is_fit = True

        return self

    def evaluate(self, grid: pkde.Grid) -> np.ndarray:
        self._kde = pkde.DensityEstimation(self._data, grid=grid, device=self.info.device)
        self._kde.estimate_density(self.info.method)
        density_estimated = self._kde.get_density()

        return density_estimated

In [13]:
@dataclass
class GridSpec:
    dim: int
    n_points: int

In [14]:
@dataclass
class DataSpec:
    dim: int
    n_samples: int

In [15]:
@dataclass
class RunSpec:
    n_warmup: int = 1
    n_runs: int = 10

In [16]:
def time_block(fn):
    t0 = time.perf_counter()
    out = fn()
    t1 = time.perf_counter()

    return out, (t1 - t0)

In [17]:
def available_estimators():
    ests = []
    
    # Scipy estimators
    for n in [1, 2]:
        for m in ["scott"]:
            ests.append(ScipyKDE(n, m))
            
    # Sklearn estimators
    for n in [1, 2]:
        for m in ["scott"]:
            ests.append(SklearnKDE(n, m))

    # Statsmodels estimators
    for n in [1, 2]:
        if n == 1:
            ms = ["scott"]
            for m in ms:
                ests.append(StatsmodelsKDE1D(m))
        else:
            ms = ["normal_reference"] # we can add cross-validation later because it's really slow
            for m in ms:
                ests.append(StatsmodelsKDE2D(m))

    # KDEpy estimators
    for n in [1]: # KDEpy only has bandwidth detection for 1D
        for m in ["scott", "ISJ"]:
            ests.append(KDEpyKDE(n, m))

    # ParallelKDE estimators
    for n in [1, 2]:
        for m in ["parallelEstimator", "rotEstimator"]:
            for d in ["cpu", "cuda"]:
                ests.append(ParallelKDE(n, m, d))

    return ests

In [18]:
def run_benchmark(estimators, datasets, grids, run_spec: RunSpec = None, verbose=False):
    if run_spec is None:
        run_spec = RunSpec()

    rows = []
    for gs in tqdm(grids, desc="Iterating grids..."):
        for ds in tqdm(datasets, desc="Iterating sample sizes..."):
            if ds.dim != gs.dim:
                continue

            dim = ds.dim

            distro = get_distro(dim)
            samples = sample_distro(distro, ds.n_samples)

            for est in estimators:
                if est.info.dim != dim:
                    continue
                if verbose:
                    print(f"Running {est.name()} with: dimensions: {dim}; grid size: {gs.n_points}; sample size:{ds.n_samples}")
                
                grid = create_grid(gs.n_points, dim, device=est.info.device)
                
                # Warmups (check if estimations work correctly and compile if needed)
                failed = False
                for _ in range(run_spec.n_warmup):
                    try:
                        est.fit(samples)
                        _ = est.evaluate(grid)
                    except Exception as e:
                        print(f"ERROR: {est.name()}")
                        rows.append({
                            "package": est.info.package,
                            "method": est.info.method,
                            "device": est.info.device,
                            "dim": est.info.dim,
                            "sample_size": ds.n_samples,
                            "grid_points": gs.n_points,
                            "ok": False,
                            "error": str(e),
                        })
                        failed = True
                        break

                if failed:
                    continue

                # Timed runs
                for _ in range(run_spec.n_runs):
                    _, fit_t = time_block(lambda: est.fit(samples))
                    _, eval_t = time_block(lambda: est.evaluate(grid))
                    rows.append({
                        "package": est.info.package,
                        "method": est.info.method,
                        "device": est.info.device,
                        "dim": est.info.dim,
                        "sample_size": ds.n_samples,
                        "grid_points": gs.n_points,
                        "fit_time_s": fit_t,
                        "eval_time_s": eval_t,
                        "total_time_s": fit_t + eval_t,
                        "ok": True,
                    })

    df = pd.DataFrame(rows)
    prefer = ["package", "method", "device", "dim", "sample_size", "grid_points", "fit_time_s", "eval_time_s", "total_time_s", "ok", "error"]
    cols = [c for c in prefer if c in df.columns] + [c for c in df.columns if c not in prefer]

    return df[cols]

## Quick check

In [19]:
datasets = [
    DataSpec(1, 1000),
    DataSpec(2, 10000),
]
grids = [
    GridSpec(1, 500),
    GridSpec(2, 125),
]
ests = available_estimators()

In [20]:
results_quick = run_benchmark(ests, datasets, grids, verbose=True)

Iterating grids...:   0%|          | 0/2 [00:00<?, ?it/s]

Iterating sample sizes...:   0%|          | 0/2 [00:00<?, ?it/s]

Running scipy:scott:cpu:1D with: dimensions: 1; grid size: 500; sample size:1000
Running sklearn:scott:cpu:1D with: dimensions: 1; grid size: 500; sample size:1000
Running statsmodels:scott:cpu:1D with: dimensions: 1; grid size: 500; sample size:1000
Running kdepy:scott:cpu:1D with: dimensions: 1; grid size: 500; sample size:1000
Running kdepy:ISJ:cpu:1D with: dimensions: 1; grid size: 500; sample size:1000
Running parallelkdepy:parallelEstimator:cpu:1D with: dimensions: 1; grid size: 500; sample size:1000
Running parallelkdepy:parallelEstimator:cuda:1D with: dimensions: 1; grid size: 500; sample size:1000
Running parallelkdepy:rotEstimator:cpu:1D with: dimensions: 1; grid size: 500; sample size:1000
Running parallelkdepy:rotEstimator:cuda:1D with: dimensions: 1; grid size: 500; sample size:1000


Iterating sample sizes...:   0%|          | 0/2 [00:00<?, ?it/s]

Running scipy:scott:cpu:2D with: dimensions: 2; grid size: 125; sample size:10000
Running sklearn:scott:cpu:2D with: dimensions: 2; grid size: 125; sample size:10000
Running statsmodels:normal_reference:cpu:2D with: dimensions: 2; grid size: 125; sample size:10000
Running parallelkdepy:parallelEstimator:cpu:2D with: dimensions: 2; grid size: 125; sample size:10000
Running parallelkdepy:parallelEstimator:cuda:2D with: dimensions: 2; grid size: 125; sample size:10000
Running parallelkdepy:rotEstimator:cpu:2D with: dimensions: 2; grid size: 125; sample size:10000
Running parallelkdepy:rotEstimator:cuda:2D with: dimensions: 2; grid size: 125; sample size:10000


In [21]:
results_quick

,package,method,device,dim,sample_size,grid_points,fit_time_s,eval_time_s,total_time_s,ok
0,scipy,scott,cpu,1,1000,500,0.000298,0.015955,0.016253,True
1,scipy,scott,cpu,1,1000,500,0.000296,0.015723,0.016019,True
2,scipy,scott,cpu,1,1000,500,0.000331,0.020616,0.020947,True
3,scipy,scott,cpu,1,1000,500,0.000347,0.016639,0.016986,True
4,scipy,scott,cpu,1,1000,500,0.000323,0.016437,0.016760,True
...,...,...,...,...,...,...,...,...,...,...
155,parallelkdepy,rotEstimator,cuda,2,10000,125,0.000002,0.007990,0.007991,True
156,parallelkdepy,rotEstimator,cuda,2,10000,125,0.000002,0.005595,0.005597,True
157,parallelkdepy,rotEstimator,cuda,2,10000,125,0.000002,0.005571,0.005573,True
158,parallelkdepy,rotEstimator,cuda,2,10000,125,0.000003,0.007418,0.007421,True


## Benchmark

In [19]:
specs_samples = (
    [DataSpec(1, m) for m in (100, 1000, 10000, 100000)] + [DataSpec(2, m) for m in (1000, 10000, 100000, 1000000)], [GridSpec(1, 500), GridSpec(2, 100)]
)
specs_grids = (
    [DataSpec(1, 10000), DataSpec(2, 100000)], [GridSpec(1, m) for m in (100, 500, 2500)] + [GridSpec(2, m) for m in (33, 100, 300)]
)
ests = available_estimators()

### Samples benchmark

In [ ]:
results_samples = run_benchmark(ests, *specs_samples)

Iterating grids...:   0%|          | 0/2 [00:00<?, ?it/s]

Iterating sample sizes...:   0%|          | 0/8 [00:00<?, ?it/s]

Iterating sample sizes...:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
results_samples.to_csv("benchmark_samples.csv", index=False)

### Grids benchmark

In [ ]:
results_grids = run_benchmark(ests, *specs_grids)

In [ ]:
results_grids.to_csv("benchmark_grids.csv", index=False)